In [2]:
import json
import requests
import logging, os, re, datetime

from arcgis.gis import GIS
from arcgis.mapping import WebMap
from arcgis.mapping import WebScene
from arcgis.features import FeatureLayerCollection
from arcgis.features import FeatureLayer
from arcgis.features import FeatureSet

import time
import scripts.portalmanager as pm

In [3]:
portals_csv = '../Conf/portal_connect_deets.csv'

usr_args_from_gis_alias_name = 'ISP_OAUTH2' # 'Geoportal_DEV' #'ISP_OAUTH2' #'Geoportal_DEV' #
usr_args_to_gis_alias_name = 'DIGITAL_DEV' # 'Geoportal_DEV'
migration_name = "ISP_to_DIGITAL_DEV"  # "portaldev_to_digital_dev" #"ISP_to_DIGITAL_DEV" #
migrate_itemid = '476b7a011eb34f1dad03ae5e76a2dc90'#connect to source and target GIS env

today = datetime.datetime.now().strftime('%d%m%Y')
data_folder = '../migration_data'
source = pm.portal_connect(portals_csv, usr_args_from_gis_alias_name, username=None, auth_type='Pro')
target = pm.portal_connect(portals_csv, usr_args_to_gis_alias_name, username=None, auth_type='OAuth 2.0')

Connected to https://ispgeospatial.com/portal/ as Santiago.Patino@aurecongroup.com (role: org_admin)
Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://ppgeospatial-dev.aurecongroup.digital/arcgis/sharing/rest/oauth2/authorize?response_type=code&client_id=ynHaDt7Xb3r9drJg&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=1i7njGzRH18uiRcukeDRq2ZLYfBxDm&allow_verification=false


C:\Users\santiago.patino\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ppgeospatial-dev.aurecongroup.digital'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://ppgeospatial-dev.aurecongroup.digital/arcgis/sharing/rest/oauth2/authorize?response_type=code&client_id=ynHaDt7Xb3r9drJg&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=zR8DeOFYWXevmAM1xgtkN2qNyKOXbN&allow_verification=false


C:\Users\santiago.patino\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ppgeospatial-dev.aurecongroup.digital'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Connected to https://ppgeospatial-dev.aurecongroup.digital/arcgis/home/ as Santiago.Patino@aurecongroup.com (role: org_publisher)


In [20]:
source_gis =source
target_gis = target
source_gis

GIS @ https://ispgeospatial.com/portal/ version:10.3

In [19]:
target_gis

GIS @ https://ppgeospatial-dev.aurecongroup.digital/arcgis/home/ version:2023.2

In [5]:
SaveLogsTo = '../Logs'
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

logFileName = datetime.datetime.now().strftime('%Y-%m-%d %H-%M-%S')
fileHandler = logging.handlers.RotatingFileHandler('{}/{}.log'.format(SaveLogsTo, logFileName), maxBytes=100000, backupCount=5)
formatter = logging.Formatter('%(asctime)s %(levelname)s %(relativeCreated)d \n%(filename)s %(module)s %(funcName)s %(lineno)d \n%(message)s\n')
fileHandler.setFormatter(formatter)
logger.addHandler(fileHandler)
logger.info('Script Starting at {}'.format(str(datetime.datetime.now())))

INFO:__main__:Script Starting at 2024-02-28 14:40:56.319554


In [6]:
def update_capabilities(url, token):
    index = url.find('services/')
    admin_url = url[:index] + "admin/" + url[index:]
    cap_update_url = admin_url + "/updateDefinition"
    json = str("{'capabilities': 'Query'}")
    params = {"token": token, "updateDefinition": json, 'f':'json'}
    requests.post(cap_update_url, params)

In [7]:
def correct_layer_ids(url, token, ids_from, ids_to):
    # CHange user URL to Admin URL
    url = url.replace("rest/services","admin/services")
    url = url.replace("/FeatureServer",".FeatureServer")
    # Get the service defintion
    payload = {"token": token,'f': 'json'}
    #head = {"Content-type": "application/json;charset=UTF-8", "Accept": "text/plain"}
    response = requests.post(url, data = payload)

    if response.status_code == 200:
        service_defn = response.json()
        print("Get the service defintion request succeeded")
    else:
        print("Get the service defintion request failed with status code:", response.status_code)
    # Grab the current deifntion of the layers dictionary
    layers = service_defn["jsonProperties"]["layers"]
    # Reset the layerIds property
    service_defn["jsonProperties"]["layerIds"] = ids_from
    # Create the subsitutue dictionary
    new_layers = {}
    
    # Copy each defintion but assign it to the correct id
    for i in range(len(ids_from)):
        from_id = ids_from[i]
        to_id = ids_to[i]
        new_layers[from_id] = layers[to_id]
        new_layers[from_id]["id"] = int(from_id)
    # Assign back to service defintion
    service_defn["jsonProperties"]["layers"] = new_layers
    
    # Now update service via edit method
    payload = {'token': token, 'f': 'json', "service": json.dumps(service_defn)}
    #head = {"Content-type": "application/x-www-form-urlencoded", "Accept": "text/plain"}
    response = requests.post(url+"/edit", data = payload)

    if response.status_code == 200:
        print("Edit layer id request succeeded")
    else:
        print("Edit layer id request failed with status code:", response.status_code)

In [8]:
def move_to_folder(itemid, folderid, token):
    url = target_gis.url + "/sharing/rest/content/users/" + target_gis.properties.user.username + "/moveItems"
    #url = "https://air-arcgis.aurecongroup.digital/arcgis/sharing/rest/content/users/aureconairpublisher/moveItems"
    payload = {'token': token, 'f': 'json', 'items': itemid, 'folder': folderid}
    response = requests.post(url, data = payload)
    if response.status_code == 200:
        print(response.json())
    else:
        print("Request failed with status code:", response.status_code)

In [9]:
def append_data(url, text, token):
    print("starting append")
    ad_url = url + "/addFeatures"
    
    payload = {'token': token, 'f': 'pjson', 'features': text}
    response = requests.post(ad_url, data = payload)
    if response.status_code == 200:
        print("Successfully added features")
    else:
        print("Added features request failed with status code:", response.status_code)

In [10]:
def max_record_count(url, token):
    # Change user URL to Admin URL
    url = url.replace("rest/services","admin/services")
    url = url.replace("/FeatureServer",".FeatureServer")
    # Get the service defintion
    payload = {"token": token,'f': 'json'}
    response = requests.post(url, data = payload)
    if response.status_code == 200:
        service_defn = response.json()
        print("Get the service defintion request succeeded")
    else:
        print("Get the service defintion request failed with status code:", response.status_code)
    # Grab the current deifntion of the layers dictionary
    layers = service_defn["jsonProperties"]["layers"]
    # Reset the maxRecordCount property
    service_defn["jsonProperties"]["maxRecordCount"] = 2000
    for layer in layers:
        print(layer)
        layers[layer]['maxRecordCount'] = 2000
    
    # Now update service via edit method
    payload = {'token': token, 'f': 'json', "service": json.dumps(service_defn)}
    response = requests.post(url+"/edit", data = payload)
    if response.status_code == 200:
        print("Edit layer maxRecordCount request succeeded")
    else:
        print("Edit layer maxRecordCount request failed with status code:", response.status_code)

In [11]:
def search_slpk(url, text, token):
    srch_url = url + "/sharing/rest/search"
    payload = {'token': token, 'f': 'pjson', 'q': text}
    response = requests.post(srch_url, data = payload)
    slpk_list = []
    if response.status_code == 200:
        for slpk in response.json()['results']:
            if slpk['type'] == "Scene Package":
                slpk_list.append(slpk['title'])
        return slpk_list
    
    else:
        print("Request failed with status code:", response.status_code)

In [12]:
def create_empty_service(svc_name, svc_disc, svc_tags, svc_snippet, svc_keyword):            
    create_parameters = {
                "name": svc_name,
                "description": svc_disc,
                "capabilities": "Editing,Query,Update,Delete",#need to turn off the editing, update, Delete as final publication function
                "spatialReference": {"wkid": "102100"},
                "zDefault": 0,
                "properties": {
                    "path": "",
                    "description": "",
                    "copyright": ""
                }
    }
    service_item = target_gis.content.create_service(name = svc_name, 
                                                     service_type = 'featureService', 
                                                     create_params = create_parameters, 
                                                     tags = svc_tags, 
                                                     snippet = svc_snippet, 
                                                     item_properties = {'typeKeywords':svc_keyword})
    return service_item

In [13]:
def layer_json_prepare (layer_item, rec_start, target_svc, ids_from, ids_to, target_gis):
    src_url = layer_item.url
    logger.info('start layer url at {}'.format(str(datetime.datetime.now())))
    logger.info('layer url is {}'.format(src_url))
    src_fl_id = src_url.split("/")[-1]
    trg_fl_id = "/" + str(rec_start)
    print(src_url)
    ids_from.append(src_fl_id)
    ids_to.append(str(rec_start))
    json_str = json.loads(f'{layer_item.properties}')
    del json_str['serviceItemId']
    del json_str['sourceSpatialReference']
    temp_json = {'layers':[]}
    temp_json["layers"].append(json_str)
    print("adding definition")
    logger.info('adding definition at {}'.format(str(datetime.datetime.now())))
    target_svc.manager.add_to_definition(temp_json)
    time.sleep(10)
    new_fl = FeatureLayer(target_svc.url + trg_fl_id, gis = target_gis)
    print("start to query")
    
    src_fl = FeatureLayer(src_url, gis = source_gis)
    #src_fset = layer_item.query("1=1")
    src_fset = src_fl.query("1=1")
    src_fset_df = src_fset.sdf
    #print(src_fset_df)
    rename = {}
    columns = src_fset_df.columns
    for column in columns:
        #print(column)
        if column != "SHAPE":
            rename[column] = column.lower()#need to be carefull about the definition query, popup etc as final publication function.

    src_fset_df.rename(columns = rename, inplace = True)
    #print(src_fset_df)
    print(src_fset_df.index)
    batch_size = 100
    batches = [group for _, group in src_fset_df.groupby(src_fset_df.index // batch_size)]
    #print(batches)
    print("starting batching")
    logger.info('starting batching at {}'.format(str(datetime.datetime.now())))
    for i, batch_df in enumerate(batches):

        #final_fs = FeatureSet.from_dataframe(batch_df.drop(columns=['assessment_date']))
        final_fs = FeatureSet.from_dataframe(batch_df)
        #print(final_fs.features)
        result = new_fl.edit_features(adds = final_fs)
        #result = new_fl.edit_features(adds = final_fs, use_global_ids = True)
        #time.sleep(0.1)
    time.sleep(5)
    return ids_from, ids_to
    print("end batching")

In [14]:
def update_popup_labels (json_string):

    for layer in json_string:
        print(layer['id'])
        if 'popupInfo' not in layer:
            print("not in")
        else:
            print("in")
            #print(layer['popupInfo'])
            #layer['popupInfo']['popupElements']:
            #print(layer['popupInfo']['fieldInfos'])
            if 'popupElements' in layer['popupInfo']:
                print ("has popupElements ")
                for field in layer['popupInfo']['popupElements']:
                    #print(field)
                    if 'fieldInfos' in field:
                        field_list = field['fieldInfos']
                        print ("has fieldInfos in popupElements ")
                        for fname in field_list:
                            #print(fname['fieldName'].lower())

                            data = fname['fieldName'].lower().replace("shape.starea()",	"SHAPE__Area").replace("shape.stlength()", "SHAPE__Length")
                            new_field = {'fieldName':data}
                            fname.update(new_field)
                            #print (fname)
                    else:
                        pass
                    if 'mediaInfos' in field:
                        print ("has mediaInfos in popupElements ")
                        #print(field['mediaInfos'][0]['value']['fields'])
                        field_list = field['mediaInfos'][0]['value']['fields']
                        data = [l.lower() for l in field_list]
                        #print(field_list)
                        print(data)
                        #print(field['mediaInfos'][0]['value'])
                        new_field = {'fields':data}
                        #print(new_field)
                        field['mediaInfos'][0]['value'].update(new_field)
                        #print(field['mediaInfos'][0]['value'])
                    else:
                        pass

            if 'expressionInfos' in layer['popupInfo']:
                print ("has expressionInfos")
                expressions = layer['popupInfo']['expressionInfos']
                if len(expressions) != 0:
                    for expression in expressions:
                        #print (expression['expression'])
                        if expression['expression'].find("Shape.STLength()") != -1 or expression['expression'].find("Shape.STArea()") != -1:
                            #print("Contains given substring")
                            new_exp = {'expression': expression['expression'].lower().replace("shape.starea()","SHAPE__Area").replace("shape.stlength()", "SHAPE__Length")}
                            expression.update(new_exp)
                            #print(expression)
                        else:
                            #print("Doesn't contains given substring")
                            #print(expression['expression'].lower())
                            new_exp = {'expression': expression['expression'].lower()}
                            expression.update(new_exp)
                            #print(expression)
            if 'layerDefinition' in layer: 
                if 'drawingInfo' in layer['layerDefinition']:
                    if 'labelingInfo' in layer['layerDefinition']['drawingInfo']:
                        label_info = layer['layerDefinition']['drawingInfo']['labelingInfo']
                        for field in label_info:
                            print(field)
                            if 'labelExpression' in field:
                                field['labelExpression'] = field['labelExpression'].lower().replace("shape.starea()","SHAPE__Area").replace("shape.stlength()", "SHAPE__Length")
                            if 'labelExpressionInfo' in field:
                                field['labelExpressionInfo']['expression'] = field['labelExpressionInfo']['expression'].lower().replace("shape.starea()","SHAPE__Area").replace("shape.stlength()", "SHAPE__Length")
                            

    return json_string

In [15]:
#source_gis.url
def related_item_id (scene_lyr_id,url,token):
    rel_trace_url = url + "sharing/rest/content/items/" + scene_lyr_id + "/relatedItems"
    print (rel_trace_url)
    payload = {'token': token, 'f': 'json', 'relationshipType': "Service2Data", 'direction': 'forward'}
    response = requests.post(rel_trace_url, data = payload)
    if response.status_code == 200:
        print("Request succeed with status code:", response.status_code)
        return response.json()['relatedItems'][0]['id']
    
    else:
        print("Request failed with status code:", response.status_code)
    

In [16]:
#get service id and get service name
def get_item_id(url, token):
    print(url)
    url_elements = url.split("/")
    if 'FeatureServer' in url_elements:
        index = url.find('FeatureServer/')
        svc_url = url[:index] + "FeatureServer"
        payload = {"token": token,'f': 'json'}  
        response = requests.post(svc_url, data = payload)
        item_id = response.json()["serviceItemId"]
        item_service_name = url_elements[-3]
        return item_id, item_service_name
    elif 'SceneServer' in url_elements:
        index = url.find('SceneServer/')
        svc_url = url[:index] + "SceneServer"
        payload = {"token": token,'f': 'json'}  
        response = requests.post(svc_url, data = payload)
        item_id = response.json()["serviceItemId"]
        item_service_name = response.json()["serviceName"]
        return item_id, item_service_name
    else:
        pass

In [27]:
def process_map(source_gis, webmap_itemid):
    print("-"*10 + webmap_itemid + "-"*10)
    print('Map Starting at {}'.format(str(datetime.datetime.now())))
    #logger.info('Map ID is {}'.format(str(dair_map_id))
    report_rev_id = 1253
    report_rev_name = "_Rev" + str(report_rev_id)
    src_map = source_gis.content.get(webmap_itemid)
    src_map_title = src_map.title
    src_map_type = src_map.type
    src_map_group = src_map.shared_with['groups'][0] 
    src_map_owner = src_map.owner
    src_map_folder_id = src_map.ownerFolder
    src_map_group_id = src_map_group.groupid
    src_map_group_name = src_map_group.title
    src_user = source_gis.users.search(src_map_owner)
    src_folders = src_user[0].folders
    src_folder_name = [folder for folder in src_folders if src_map_folder_id in folder['id']][0]['title']
    print ("Map title: " + src_map_title)
    print ("map type: " + src_map_type)
    print ("owner: " + src_map_owner)
    print ("folder id: " + src_map_folder_id)
    print ("folder name: " + src_folder_name)
    print ("group id: " + src_map_group_id)
    print ("group name: " + src_map_group_name)
    print ("publication rev: Rev" + str(report_rev_id))
    if src_map_type == "Web Map":
        src_wmp = WebMap(src_map)
        itemslist = []
        item_name_list =[]
        for src_layer in src_wmp.layers:
            item_id, item_svc_name = get_item_id(src_layer.url, source_gis._con.token)
            if item_id not in itemslist:
                
                itemslist.append(item_id)
                item_name_list.append([item_id, item_svc_name])
                
        #print([*set(itemslist)])
        #itemslist_uniq = [*set(itemslist)]
        print(item_name_list)
    

In [28]:
webmap_itemid = '476b7a011eb34f1dad03ae5e76a2dc90'
process_map(source_gis, webmap_itemid)

----------476b7a011eb34f1dad03ae5e76a2dc90----------
Map Starting at 2024-02-28 14:56:07.689112


INFO:arcgis.gis._impl._portalpy:Searching users (q=Santiago.Patino@aurecongroup.com accountid:0123456789ABCDEF, start=1, num=100)
INFO:arcgis.gis._impl._portalpy:getting user folders and items


Map title: test migration map
map type: Web Map
owner: Santiago.Patino@aurecongroup.com
folder id: f0c0316bc625458488a0f6dae13e9c09
folder name: test to migrate
group id: 70bec880f9bb430f90a0bacd02bb25d8
group name: test_migration_content_group
publication rev: Rev1253


AttributeError: 'PropertyMap' instance has no attribute 'url'

In [17]:
# air_map_list = ['476b7a011eb34f1dad03ae5e76a2dc90']
webmap_itemid = '476b7a011eb34f1dad03ae5e76a2dc90'
with webmap_itemid :

    print("-"*10 + webmap_itemid + "-"*10)
    print('Map Starting at {}'.format(str(datetime.datetime.now())))
    #logger.info('Map ID is {}'.format(str(dair_map_id))
    report_rev_id = 1253
    report_rev_name = "_Rev" + str(report_rev_id)
    src_map = source_gis.content.get(webmap_itemid)
    src_map_title = src_map.title
    src_map_type = src_map.type
    src_map_group = src_map.shared_with['groups'][0]
    src_map_owner = src_map.owner
    src_map_folder_id = src_map.ownerFolder
    src_map_group_id = src_map_group.groupid
    src_map_group_name = src_map_group.title
    src_user = source_gis.users.search(src_map_owner)
    src_folders = src_user[0].folders
    src_folder_name = [folder for folder in src_folders if src_map_folder_id in folder['id']][0]['title']
    print ("Map title: " + src_map_title)
    print ("map type: " + src_map_type)
    print ("owner: " + src_map_owner)
    print ("folder id: " + src_map_folder_id)
    print ("folder name: " + src_folder_name)
    print ("group id: " + src_map_group_id)
    print ("group name: " + src_map_group_name)
    print ("publication rev: Rev" + str(report_rev_id))
    #create new webmap or webscene in exhibition tenancy
    #webmap use adding function,feature layer using creating service and append json method, webscene use addItem api to make a copy
    if src_map_type == "Web Map":
        src_wmp = WebMap(src_map)
        itemslist = []
        item_name_list =[]
        for src_layer in src_wmp.layers:
            item_id, item_svc_name = get_item_id(src_layer.url, source_gis._con.token)
            if item_id not in itemslist:
                
                itemslist.append(item_id)
                item_name_list.append([item_id, item_svc_name])
                
        #print([*set(itemslist)])
        #itemslist_uniq = [*set(itemslist)]
        print(item_name_list)
        for src_item_id in item_name_list:
            print(src_item_id)
            src_fl = source_gis.content.get(src_item_id[0])
            print(src_fl.title)
            new_svc_name = src_item_id[1] + report_rev_name
            print(new_svc_name)
            name_available = target_gis.content.is_service_name_available(service_name = new_svc_name, service_type = 'featureService')
            if name_available == True:
                print ("feature layer is not existing, create a new blank service now...")
                source_kw = "source-" + src_item_id[0]
                empty_service_item = create_empty_service(new_svc_name, src_fl.description, src_fl.tags, src_fl.snippet, source_kw)
                print("New one has been created...")


                src_flc = FeatureLayerCollection.fromitem(src_fl)
                trg_flc = FeatureLayerCollection.fromitem(empty_service_item)
                if src_flc.properties.enableZDefaults == True:
                    print("has z...")
                    update_dict = {"enableZDefaults": True}
                    trg_flc.manager.update_definition(update_dict)
                    print("enabled z...")
                else:
                    pass
                print("Start to appending layers...")
                ids_from = []
                ids_to = []
                rec = -1
                for layer in src_fl.layers:
                    rec += 1
                    ids_from, ids_to = layer_json_prepare (layer, rec, trg_flc, ids_from, ids_to,target_gis)

                    print("data appended")
                    #print(src_fset)         

                print("Start correcting layer ids...")
                correct_layer_ids(empty_service_item.url, target_gis._con.token, ids_from, ids_to)
                print("Correcting layer ids done...")
                print ("Start updating capability...")
                update_capabilities(empty_service_item.url, target_gis._con.token)
                print ("cloning and updating capability have done...")
                max_record_count(empty_service_item.url, target_gis._con.token)
            else:
                #empty_service_item = target_gis.content.get('789ff05ffd664cca8d4f8ab9c235c707')
                empty_service_item = []
                print ("feature layer is exiting, no proceed to clone")
    elif src_map_type == "Web Scene":
        print ("This map is {map_type}, will clone it as new one now...".format(map_type = src_map_type))

        sr_webscene_obj = WebScene(src_map)
        sr_webscene_lyrs = sr_webscene_obj['operationalLayers']
        proc_list = []
        for layer in sr_webscene_lyrs:
            item_id, item_svc_name = get_item_id(layer['url'], source_gis._con.token)

            if layer['layerType'] == 'ArcGISSceneServiceLayer':
                empty_service_item = []


                print ("Scene layer ID : " + item_id)
                print ("Scene Layer Service Name: " + item_svc_name)

                rel_slpk_id = related_item_id(item_id,source_gis.url,source_gis._con.token)

                src_slpk_item = source_gis.content.get(rel_slpk_id)
                print ("SLPK Name: " + src_slpk_item.title)
                new_slpk_name = src_slpk_item.title + report_rev_name

                trg_slpk_list = search_slpk(target_gis.url, new_slpk_name, target_gis._con.token)
                #print(trg_slpk_list)
                if new_slpk_name not in trg_slpk_list:
                    print ("SLPK is not existing, cloning it now")
                    new_slpk_clone = target_gis.content.clone_items(items = [src_slpk_item], folder = src_folder_name, search_existing_items = True, copy_data = True, preserve_item_id = False)
                    print (new_slpk_clone[0].id)
                    source_kw = "source-" + item_id
                    new_slpk_properties = {'title': new_slpk_name,'typeKeywords':source_kw}
                    new_slpk_clone[0].update(item_properties = new_slpk_properties)
                    print ("slpk cloning is done")
                    print ("publishing slpk")
                    publish_parameters = {'name':item_svc_name + report_rev_name}
                    new_scene_lyr = new_slpk_clone[0].publish(publish_parameters, file_type = "scenepackage",
                                                     output_type = "sceneservice", build_initial_cache = True)

                    print ("new scene layer has published")
                else:
                    print ("SLPK is exiting, no need to clone")

            elif layer['layerType'] == 'ArcGISFeatureLayer':
                #item_id = get_item_id(layer['url'], source_gis._con.token)
                if item_id not in proc_list:
                    print ("2D Layer ID: " + item_id)
                    print ("2D Layer Name: " + item_svc_name)
                    src_fl = source_gis.content.get(item_id)
                    new_svc_name = item_svc_name + report_rev_name
                    name_available = target_gis.content.is_service_name_available(service_name = new_svc_name, service_type = 'featureService')
                    if name_available == True:
                        source_kw = "source-" + item_id                
                        empty_service_item = create_empty_service(new_svc_name, src_fl.description, src_fl.tags, src_fl.snippet, source_kw)
                        print("New one has been created...")
                        src_flc = FeatureLayerCollection.fromitem(src_fl)
                        trg_flc = FeatureLayerCollection.fromitem(empty_service_item)
                        if src_flc.properties.enableZDefaults == True:
                            print("has z...")
                            update_dict = {"enableZDefaults": True}
                            trg_flc.manager.update_definition(update_dict)
                            print("enabled z...")
                        else:
                            pass
                        print("Start to appending layers...")
                        ids_from = []
                        ids_to = []
                        rec = -1
                        for layer in src_fl.layers:
                            rec += 1
                            ids_from, ids_to = layer_json_prepare (layer, rec, trg_flc, ids_from, ids_to,target_gis)
                            #print(json_str)
                            #append_data(new_fl_url,json.dumps(json_body),target_gis._con.token)
                            print("data appended")
                            #print(src_fset)
                        print(ids_from)
                        print(ids_to)

                        print("Appending layers done...")
                        #print(ids_from)
                        #print(ids_to)
                        print("Start correcting layer ids...")
                        correct_layer_ids(empty_service_item.url, target_gis._con.token, ids_from, ids_to)
                        print("Correcting layer ids done...")
                        print ("Start updating capability...")
                        update_capabilities(empty_service_item.url, target_gis._con.token)
                        print ("cloning and updating capability have done...")
                        max_record_count(empty_service_item.url, target_gis._con.token)

                    else:
                        
                        empty_service_item = []
                        
                        print ("feature layer is exiting, no proceed to clone")
                    proc_list.append(item_id)



    else:
        pass
    print(empty_service_item)
    #check webmap is existing
    new_map_name = src_map_title + report_rev_name
    trg_check_wmp = target_gis.content.search(query = src_map.id)
    print(len(trg_check_wmp))
    if len(trg_check_wmp)!= 0:
        print ("this maps is existing")
        new_item_clone = []
    else:
        if src_map_type == "Web Scene":
            new_item_clone = target_gis.content.clone_items(items = [src_map], folder = src_folder_name, search_existing_items = True, preserve_item_id = True)
            new_item_clone[0].update(item_properties = {'title':new_map_name})
        else:
            new_item_clone = target_gis.content.clone_items(items = [src_map], folder = src_folder_name, search_existing_items = True, preserve_item_id = True)
            new_item_clone[0].update(item_properties = {'title':new_map_name})


    #update map layer urls
    if len(trg_check_wmp)== 0:
        if src_map_type == "Web Map":
            wmp_json = new_item_clone[0].get_data()
            #print(map_json1)
            map_json1 = wmp_json['operationalLayers']
            json_str = update_popup_labels(map_json1)
            item_properties = {"text": json.dumps(wmp_json)}
            new_item_clone[0].update(item_properties = item_properties)
            new_wmp = WebMap(new_item_clone[0])
            print(empty_service_item)
            if len(empty_service_item) != 0:
                for layer in new_wmp.layers:
                    #print(layer.url)
                    lry_url = layer.url[:layer.url.rfind('/')]
                    new_fl_lnk = empty_service_item.url
                    new_url = layer.url.replace(lry_url, new_fl_lnk)
                    layer.update(url = new_url, itemId = empty_service_item.id)
                new_wmp.update()
            print("New webmap has update ")
        elif src_map_type == "Web Scene":
            webscene_string = WebScene(src_map)
            map_json1 = webscene_string['operationalLayers']
            json_str = update_popup_labels(map_json1)
            for layer in map_json1:
                print (layer['url'])
                ft_title = layer['url'].split('/')
                if layer['layerType'] == 'ArcGISSceneServiceLayer':
                    index = layer['url'].find('Hosted/')
                    admin_url = layer['url'][:index] + ft_title[6] +'/' + ft_title[7]
                    target_url = target_gis.url + "/rest/services/"+ ft_title[6] +'/'+ ft_title[7] + report_rev_name
                    new_url = layer['url'].replace(admin_url, target_url)
                    layer['url'] = new_url
                elif layer['layerType'] == 'ArcGISFeatureLayer':
                    index = layer['url'].find('services/')
                    admin_url = layer['url'][:index] + ft_title[5] + '/' + ft_title[6] +'/'+ft_title[7]
                    print ("test:" + admin_url)
                    target_url = target_gis.url + "/rest/services/Hosted/"+ ft_title[7] + report_rev_name
                    print (target_url)
                    new_url = layer['url'].replace(admin_url, target_url)
                    print (new_url)
                    layer['url'] = new_url
                else:
                    pass
                print (layer['url'])
            #json_str = update_popup_labels(map_json1)
            item_properties = {"text": json.dumps(webscene_string)}
            #print (item_properties)
            new_item_clone[0].update(item_properties = item_properties)
    else:
        pass

    #move item to folder
    new_folder = target_gis.content.create_folder(src_folder_name)
    trg_user = target_gis.users.search("aureconairpublisher")
    print(new_folder)
    if new_folder == None:
        #print('yes')
        trg_folders = trg_user[0].folders
        print(trg_folders)
        new_folder_id = trg_folders[0]['id']
    else:
        new_folder_id = new_folder['id']
    if len(empty_service_item) != 0:
        move_to_folder(empty_service_item.id, new_folder_id, target_gis._con.token)
    else:
        print("no need to move item")
    #creation a group
    target_groups = target_gis.groups.search(query = src_map_group_name)
    if len(target_groups) == 0:
        print ("Creating new group...")
        new_group = target_gis.groups.create(title = src_map_group_name, tags = "", description = "", access = "private")
        print(new_group)
        target_group_id = new_group.id
        print ("new group has been created")
    else:
        print ("Group existing. no proceed")
        target_group_id = target_groups[0].id
    #share item to group
    target_user = target_gis.users.search('aureconairpublisher')[0]
    folder_items = target_user.items(folder = src_folder_name)
    print(folder_items)
    #target_group_id = target_gis.groups.search(query = src_map_group_name)[0]
    print(target_group_id)
    for item in folder_items:
        item.share(org = False, groups = target_group_id)
    print ("items have been shared to the group")
    #checking user from source portal gourp
    #usersManager = arcgis.gis.UserManager(target_gis)
    search_src_gp = source_gis.groups.search(query='id: ' + src_map_group_id)
    print(search_src_gp[0].get_members())
    src_user_list = search_src_gp[0].get_members()['users']
    src_grp_owner = search_src_gp[0].get_members()['admins']
    trg_user_list = target_groups[0].get_members()['users']
    for o in src_grp_owner:
        src_user_list.append(o)
    print(src_user_list)
    role_mgr = target_gis.users.roles
    org_roles = role_mgr.all()

    for role in org_roles:
        print(f"{role.name:25}{role.role_id}")
        if role.name =='Aurecon Air Viewer':
            role_id = role.role_id
        else:
            pass
    print(role_id)
    #adding users to site
    for src_user in src_user_list:
        if src_user != 'eia.publisher' and src_user != 'siteadmin' and src_user not in trg_user_list:
            #print(src_user)
            search_trg_users = target_gis.users.search(query = src_user)
            if len(search_trg_users) != 0:
                print(search_trg_users[0].username + "is existing")
                target_groups[0].add_users(search_trg_users[0])
                print("user added to the group")

            else:
                search_src_users = source_gis.users.search(query = src_user)
                print("create user" + src_user)
                #print(search_src_users[0].idpUsername)
                new_user = target_gis.users.create(username = search_src_users[0].username,
                                                   password = '',
                                                   firstname = search_src_users[0].firstName,
                                                   lastname = search_src_users[0].lastName,
                                                   email = search_src_users[0].email,
                                                   description = search_src_users[0].description,
                                                   idp_username = search_src_users[0].idpUsername,
                                                   role = role_id,
                                                   level = 1,
                                                   user_type = 'viewerUT',
                                                   provider = 'enterprise')
                new_user.update(search_src_users[0].access)
                target_groups[0].add_users(new_user)

        else:
            for trg_user in trg_user_list:
                if trg_user != 'eia.publisher' and trg_user != 'siteadmin' and trg_user not in src_user_list:
                    target_groups[0].remove_users(trg_user)


    print("all done")

----------476b7a011eb34f1dad03ae5e76a2dc90----------
Map Starting at 2024-02-28 14:41:06.696012


IndexError: list index out of range

In [ ]:
air_map_list = ['476b7a011eb34f1dad03ae5e76a2dc90']
for air_map_id in air_map_list:
    print("-"*10 + air_map_id + "-"*10)
    print('Map Starting at {}'.format(str(datetime.datetime.now())))
    #logger.info('Map ID is {}'.format(str(dair_map_id))
    report_rev_id = 1253
    report_rev_name = "_Rev" + str(report_rev_id)
    src_map = source_gis.content.get(air_map_id)
    src_map_title = src_map.title
    src_map_type = src_map.type
    src_map_group = src_map.shared_with['groups'][0]
    src_map_owner = src_map.owner
    src_map_folder_id = src_map.ownerFolder
    src_map_group_id = src_map_group.groupid
    src_map_group_name = src_map_group.title
    src_user = source_gis.users.search(src_map_owner)
    src_folders = src_user[0].folders
    src_folder_name = [folder for folder in src_folders if src_map_folder_id in folder['id']][0]['title']
    print ("Map title: " + src_map_title)
    print ("map type: " + src_map_type)
    print ("owner: " + src_map_owner)
    print ("folder id: " + src_map_folder_id)
    print ("folder name: " + src_folder_name)
    print ("group id: " + src_map_group_id)
    print ("group name: " + src_map_group_name)
    print ("publication rev: Rev" + str(report_rev_id))
    #create new webmap or webscene in exhibition tenancy
    #webmap use adding function,feature layer using creating service and append json method, webscene use addItem api to make a copy
    if src_map_type == "Web Map":
        src_wmp = WebMap(src_map)
        itemslist = []
        item_name_list =[]
        for src_layer in src_wmp.layers:
            item_id, item_svc_name = get_item_id(src_layer.url, source_gis._con.token)
            if item_id not in itemslist:
                
                itemslist.append(item_id)
                item_name_list.append([item_id, item_svc_name])
                
        #print([*set(itemslist)])
        #itemslist_uniq = [*set(itemslist)]
        print(item_name_list)
        for src_item_id in item_name_list:
            print(src_item_id)
            src_fl = source_gis.content.get(src_item_id[0])
            print(src_fl.title)
            new_svc_name = src_item_id[1] + report_rev_name
            print(new_svc_name)
            name_available = target_gis.content.is_service_name_available(service_name = new_svc_name, service_type = 'featureService')
            if name_available == True:
                print ("feature layer is not existing, create a new blank service now...")
                source_kw = "source-" + src_item_id[0]
                empty_service_item = create_empty_service(new_svc_name, src_fl.description, src_fl.tags, src_fl.snippet, source_kw)
                print("New one has been created...")


                src_flc = FeatureLayerCollection.fromitem(src_fl)
                trg_flc = FeatureLayerCollection.fromitem(empty_service_item)
                if src_flc.properties.enableZDefaults == True:
                    print("has z...")
                    update_dict = {"enableZDefaults": True}
                    trg_flc.manager.update_definition(update_dict)
                    print("enabled z...")
                else:
                    pass
                print("Start to appending layers...")
                ids_from = []
                ids_to = []
                rec = -1
                for layer in src_fl.layers:
                    rec += 1
                    ids_from, ids_to = layer_json_prepare (layer, rec, trg_flc, ids_from, ids_to,target_gis)

                    print("data appended")
                    #print(src_fset)         

                print("Start correcting layer ids...")
                correct_layer_ids(empty_service_item.url, target_gis._con.token, ids_from, ids_to)
                print("Correcting layer ids done...")
                print ("Start updating capability...")
                update_capabilities(empty_service_item.url, target_gis._con.token)
                print ("cloning and updating capability have done...")
                max_record_count(empty_service_item.url, target_gis._con.token)
            else:
                #empty_service_item = target_gis.content.get('789ff05ffd664cca8d4f8ab9c235c707')
                empty_service_item = []
                print ("feature layer is exiting, no proceed to clone")
    elif src_map_type == "Web Scene":
        print ("This map is {map_type}, will clone it as new one now...".format(map_type = src_map_type))

        sr_webscene_obj = WebScene(src_map)
        sr_webscene_lyrs = sr_webscene_obj['operationalLayers']
        proc_list = []
        for layer in sr_webscene_lyrs:
            item_id, item_svc_name = get_item_id(layer['url'], source_gis._con.token)

            if layer['layerType'] == 'ArcGISSceneServiceLayer':
                empty_service_item = []


                print ("Scene layer ID : " + item_id)
                print ("Scene Layer Service Name: " + item_svc_name)

                rel_slpk_id = related_item_id(item_id,source_gis.url,source_gis._con.token)

                src_slpk_item = source_gis.content.get(rel_slpk_id)
                print ("SLPK Name: " + src_slpk_item.title)
                new_slpk_name = src_slpk_item.title + report_rev_name

                trg_slpk_list = search_slpk(target_gis.url, new_slpk_name, target_gis._con.token)
                #print(trg_slpk_list)
                if new_slpk_name not in trg_slpk_list:
                    print ("SLPK is not existing, cloning it now")
                    new_slpk_clone = target_gis.content.clone_items(items = [src_slpk_item], folder = src_folder_name, search_existing_items = True, copy_data = True, preserve_item_id = False)
                    print (new_slpk_clone[0].id)
                    source_kw = "source-" + item_id
                    new_slpk_properties = {'title': new_slpk_name,'typeKeywords':source_kw}
                    new_slpk_clone[0].update(item_properties = new_slpk_properties)
                    print ("slpk cloning is done")
                    print ("publishing slpk")
                    publish_parameters = {'name':item_svc_name + report_rev_name}
                    new_scene_lyr = new_slpk_clone[0].publish(publish_parameters, file_type = "scenepackage",
                                                     output_type = "sceneservice", build_initial_cache = True)

                    print ("new scene layer has published")
                else:
                    print ("SLPK is exiting, no need to clone")

            elif layer['layerType'] == 'ArcGISFeatureLayer':
                #item_id = get_item_id(layer['url'], source_gis._con.token)
                if item_id not in proc_list:
                    print ("2D Layer ID: " + item_id)
                    print ("2D Layer Name: " + item_svc_name)
                    src_fl = source_gis.content.get(item_id)
                    new_svc_name = item_svc_name + report_rev_name
                    name_available = target_gis.content.is_service_name_available(service_name = new_svc_name, service_type = 'featureService')
                    if name_available == True:
                        source_kw = "source-" + item_id                
                        empty_service_item = create_empty_service(new_svc_name, src_fl.description, src_fl.tags, src_fl.snippet, source_kw)
                        print("New one has been created...")
                        src_flc = FeatureLayerCollection.fromitem(src_fl)
                        trg_flc = FeatureLayerCollection.fromitem(empty_service_item)
                        if src_flc.properties.enableZDefaults == True:
                            print("has z...")
                            update_dict = {"enableZDefaults": True}
                            trg_flc.manager.update_definition(update_dict)
                            print("enabled z...")
                        else:
                            pass
                        print("Start to appending layers...")
                        ids_from = []
                        ids_to = []
                        rec = -1
                        for layer in src_fl.layers:
                            rec += 1
                            ids_from, ids_to = layer_json_prepare (layer, rec, trg_flc, ids_from, ids_to,target_gis)
                            #print(json_str)
                            #append_data(new_fl_url,json.dumps(json_body),target_gis._con.token)
                            print("data appended")
                            #print(src_fset)
                        print(ids_from)
                        print(ids_to)

                        print("Appending layers done...")
                        #print(ids_from)
                        #print(ids_to)
                        print("Start correcting layer ids...")
                        correct_layer_ids(empty_service_item.url, target_gis._con.token, ids_from, ids_to)
                        print("Correcting layer ids done...")
                        print ("Start updating capability...")
                        update_capabilities(empty_service_item.url, target_gis._con.token)
                        print ("cloning and updating capability have done...")
                        max_record_count(empty_service_item.url, target_gis._con.token)

                    else:
                        
                        empty_service_item = []
                        
                        print ("feature layer is exiting, no proceed to clone")
                    proc_list.append(item_id)



    else:
        pass
    print(empty_service_item)
    #check webmap is existing
    new_map_name = src_map_title + report_rev_name
    trg_check_wmp = target_gis.content.search(query = src_map.id)
    print(len(trg_check_wmp))
    if len(trg_check_wmp)!= 0:
        print ("this maps is existing")
        new_item_clone = []
    else:
        if src_map_type == "Web Scene":
            new_item_clone = target_gis.content.clone_items(items = [src_map], folder = src_folder_name, search_existing_items = True, preserve_item_id = True)
            new_item_clone[0].update(item_properties = {'title':new_map_name})
        else:
            new_item_clone = target_gis.content.clone_items(items = [src_map], folder = src_folder_name, search_existing_items = True, preserve_item_id = True)
            new_item_clone[0].update(item_properties = {'title':new_map_name})


    #update map layer urls
    if len(trg_check_wmp)== 0:
        if src_map_type == "Web Map":
            wmp_json = new_item_clone[0].get_data()
            #print(map_json1)
            map_json1 = wmp_json['operationalLayers']
            json_str = update_popup_labels(map_json1)
            item_properties = {"text": json.dumps(wmp_json)}
            new_item_clone[0].update(item_properties = item_properties)
            new_wmp = WebMap(new_item_clone[0])
            print(empty_service_item)
            if len(empty_service_item) != 0:
                for layer in new_wmp.layers:
                    #print(layer.url)
                    lry_url = layer.url[:layer.url.rfind('/')]
                    new_fl_lnk = empty_service_item.url
                    new_url = layer.url.replace(lry_url, new_fl_lnk)
                    layer.update(url = new_url, itemId = empty_service_item.id)
                new_wmp.update()
            print("New webmap has update ")
        elif src_map_type == "Web Scene":
            webscene_string = WebScene(src_map)
            map_json1 = webscene_string['operationalLayers']
            json_str = update_popup_labels(map_json1)
            for layer in map_json1:
                print (layer['url'])
                ft_title = layer['url'].split('/')
                if layer['layerType'] == 'ArcGISSceneServiceLayer':
                    index = layer['url'].find('Hosted/')
                    admin_url = layer['url'][:index] + ft_title[6] +'/' + ft_title[7]
                    target_url = target_gis.url + "/rest/services/"+ ft_title[6] +'/'+ ft_title[7] + report_rev_name
                    new_url = layer['url'].replace(admin_url, target_url)
                    layer['url'] = new_url
                elif layer['layerType'] == 'ArcGISFeatureLayer':
                    index = layer['url'].find('services/')
                    admin_url = layer['url'][:index] + ft_title[5] + '/' + ft_title[6] +'/'+ft_title[7]
                    print ("test:" + admin_url)
                    target_url = target_gis.url + "/rest/services/Hosted/"+ ft_title[7] + report_rev_name
                    print (target_url)
                    new_url = layer['url'].replace(admin_url, target_url)
                    print (new_url)
                    layer['url'] = new_url
                else:
                    pass
                print (layer['url'])
            #json_str = update_popup_labels(map_json1)
            item_properties = {"text": json.dumps(webscene_string)}
            #print (item_properties)
            new_item_clone[0].update(item_properties = item_properties)
    else:
        pass

    #move item to folder
    new_folder = target_gis.content.create_folder(src_folder_name)
    trg_user = target_gis.users.search("aureconairpublisher")
    print(new_folder)
    if new_folder == None:
        #print('yes')
        trg_folders = trg_user[0].folders
        print(trg_folders)
        new_folder_id = trg_folders[0]['id']
    else:
        new_folder_id = new_folder['id']
    if len(empty_service_item) != 0:
        move_to_folder(empty_service_item.id, new_folder_id, target_gis._con.token)
    else:
        print("no need to move item")
    #creation a group
    target_groups = target_gis.groups.search(query = src_map_group_name)
    if len(target_groups) == 0:
        print ("Creating new group...")
        new_group = target_gis.groups.create(title = src_map_group_name, tags = "", description = "", access = "private")
        print(new_group)
        target_group_id = new_group.id
        print ("new group has been created")
    else:
        print ("Group existing. no proceed")
        target_group_id = target_groups[0].id
    #share item to group
    target_user = target_gis.users.search('aureconairpublisher')[0]
    folder_items = target_user.items(folder = src_folder_name)
    print(folder_items)
    #target_group_id = target_gis.groups.search(query = src_map_group_name)[0]
    print(target_group_id)
    for item in folder_items:
        item.share(org = False, groups = target_group_id)
    print ("items have been shared to the group")
    #checking user from source portal gourp
    #usersManager = arcgis.gis.UserManager(target_gis)
    search_src_gp = source_gis.groups.search(query='id: ' + src_map_group_id)
    print(search_src_gp[0].get_members())
    src_user_list = search_src_gp[0].get_members()['users']
    src_grp_owner = search_src_gp[0].get_members()['admins']
    trg_user_list = target_groups[0].get_members()['users']
    for o in src_grp_owner:
        src_user_list.append(o)
    print(src_user_list)
    role_mgr = target_gis.users.roles
    org_roles = role_mgr.all()

    for role in org_roles:
        print(f"{role.name:25}{role.role_id}")
        if role.name =='Aurecon Air Viewer':
            role_id = role.role_id
        else:
            pass
    print(role_id)
    #adding users to site
    for src_user in src_user_list:
        if src_user != 'eia.publisher' and src_user != 'siteadmin' and src_user not in trg_user_list:
            #print(src_user)
            search_trg_users = target_gis.users.search(query = src_user)
            if len(search_trg_users) != 0:
                print(search_trg_users[0].username + "is existing")
                target_groups[0].add_users(search_trg_users[0])
                print("user added to the group")

            else:
                search_src_users = source_gis.users.search(query = src_user)
                print("create user" + src_user)
                #print(search_src_users[0].idpUsername)
                new_user = target_gis.users.create(username = search_src_users[0].username,
                                                   password = '',
                                                   firstname = search_src_users[0].firstName,
                                                   lastname = search_src_users[0].lastName,
                                                   email = search_src_users[0].email,
                                                   description = search_src_users[0].description,
                                                   idp_username = search_src_users[0].idpUsername,
                                                   role = role_id,
                                                   level = 1,
                                                   user_type = 'viewerUT',
                                                   provider = 'enterprise')
                new_user.update(search_src_users[0].access)
                target_groups[0].add_users(new_user)

        else:
            for trg_user in trg_user_list:
                if trg_user != 'eia.publisher' and trg_user != 'siteadmin' and trg_user not in src_user_list:
                    target_groups[0].remove_users(trg_user)


    print("all done")

In [115]:
logger.info('finished at {}'.format(str(datetime.datetime.now())))
print("finished")

INFO:__main__:finished at 2023-11-23 13:55:48.215089


finished
